In [9]:
import numpy as np
import csv
import os

if __name__ == '__main__':
    emotions, speakers = [], []
    dataset = 'dailydialog'
    data = 'valid'
    file_base = '/nfs/users/gutianyun/EmoBias-ERC/model/datasets/dialogue_level_minibatch/' + dataset + '/' + dataset + '_' + data + '_'
    file_emotions = file_base + 'emotion.tsv'
    file_speakers = file_base + 'speakers.tsv'
    file_subtask = file_base + 'subtask013Senti_label.tsv'
    
    with open(file_emotions) as f:
        for line in f:
            content = line.strip().split('\t')[1:]
            emotions.append(content)
        
    with open(file_speakers) as f:
        for line in f:
            content = line.strip().split('\t')[1:]
            speakers.append(content)
            
    if os.path.exists(file_subtask):
        os.remove(file_subtask)
    
    
    pos_set = set(['4','6'])
    neg_set = set(['0','2','3','5'])
    neu_set = set(['1'])
    for j, conv in enumerate(speakers):
        unique_speakers = np.unique(conv)
        speaker_memo = {}
        for unique_speaker in unique_speakers:
            speaker_memo[unique_speaker] = conv.index(unique_speaker)
        
        res = []
        res.append('te_'+str(j))
        for i, curr_speaker in enumerate(conv):
            last_index = speaker_memo[curr_speaker]
            speaker_memo[curr_speaker] = i
            
            if i == last_index:
                res.append(2)
            else:
                last_emotion = emotions[j][last_index]
                curr_emotion = emotions[j][i]
                # subtask01: emotion shift
                '''
                if last_emotion == curr_emotion:
                    res.append(0)
                else:
                    res.append(1) 
                '''
               
                # subtask01: sentiment shift
                '''
                if ((last_emotion in pos_set) and (curr_emotion in pos_set)) \
                or ((last_emotion in neg_set) and (curr_emotion in neg_set)) \
                or ((last_emotion in neu_set) and (curr_emotion in neu_set)):
                    res.append(0)
                else:
                    res.append(1) 
                '''
                # subtask013: sentiment shift
                '''
                if ((last_emotion in pos_set) and (curr_emotion in pos_set)) \
                or ((last_emotion in neg_set) and (curr_emotion in neg_set)) \
                or ((last_emotion in neu_set) and (curr_emotion in neu_set)):
                    res.append(0)
                elif ((last_emotion in neg_set) and (curr_emotion not in neg_set))\
                or ((last_emotion in neu_set) and (curr_emotion in pos_set)):
                    res.append(1) # shift to pos
                else:
                    res.append(3) # shift to neg
                '''
                
        # print('res:', res)
        
        with open(file_subtask, 'a', newline='') as f_output:
            tsv_output = csv.writer(f_output, delimiter = '\t')
            tsv_output.writerow(res)
        